### Data Tables

In [8]:
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template

import plotly.express as px
import pandas as pd
import numpy as np

dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"

style = {
    'backgroundColor': 'rgb(30, 30, 30)',
    'color': 'lightgrey',
    'font-family': "Arial"
}

from dash import dash_table

app = JupyterDash(__name__, # external_stylesheets=[dbc.themes.SLATE, dbc_css]
                 )

education = pd.read_csv("../Data/states_all.csv").iloc[:, 1:]

from dash import dash_table

app.layout = html.Div(
    dash_table.DataTable(
        columns=[{"name": i, "id": i} for i in education.columns],
        data=education.to_dict("records"),
        filter_action="native",
        sort_action="native",
        export_format="csv",
        style_header=style,
        style_data=style
    )
)

app.run_server(debug=True, port=8282)


Dash app running on http://127.0.0.1:8282/


### DBC Tables

In [9]:
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template

import plotly.express as px
import pandas as pd
import numpy as np

dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"

from dash import dash_table

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SLATE, dbc_css])

education = pd.read_csv("../Data/states_all.csv").iloc[:, 1:]

app.layout = dbc.Container(
    dbc.Table.from_dataframe(
        education, 
        striped=True, 
        bordered=True, 
        hover=True,
#         color="dark",
        class_name="dbc"
    )
)

app.run_server(debug=True,port=8262)


Dash app running on http://127.0.0.1:8262/


### Conditional Callbacks

In [5]:
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template

import plotly.express as px
import pandas as pd
import numpy as np

dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SLATE, dbc_css])

style = {
    'backgroundColor': 'rgb(30, 30, 30)',
    'color': 'lightgrey',
    'font-family': "Arial"
}

load_figure_template("SLATE")

df = (
    pd.read_csv("../Data/states_all.csv")
    .iloc[:, 1:]
    .groupby("STATE", as_index=False)
    .agg({"TOTAL_EXPENDITURE": "mean"})
    .sort_values("TOTAL_EXPENDITURE", ascending=False)
)

app.layout = html.Div([
    dcc.RadioItems(
        id="OutputPicker",
        options=["bar", "table"],
        value="bar"
    ),
    html.Div(id="Output Div")    
])

@app.callback(Output("Output Div", "children"), Input("OutputPicker", "value"))
def output_generator(output_style):
    if output_style == "bar":
        output = dcc.Graph(figure=px.bar(df, x="STATE", y="TOTAL_EXPENDITURE")) 
    else:
        output = dash_table.DataTable(             
            columns=[{"name": i, "id": i} for i in df.columns],
            data=df.to_dict("records"),
            style_header=style,
            style_data=style
        )
        
    return output  

app.run_server(debug=True, port=8283)

Dash app running on http://127.0.0.1:8283/


### Chained Callbacks

In [1]:
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template

import plotly.express as px
import pandas as pd
import numpy as np

dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SLATE, dbc_css])

states_cities = {
    "California": ["Los Angeles", "San Diego", "San Francisco"],
    "Oregon": ["Bend", "Eugene", "Portland"],
    "Washington": ["Olympia", "Spokane", "Seattle"],
}

app.layout= html.Div([
    dcc.Dropdown(
        id="state-dropdown", 
        options=["California", "Oregon", "Washington"], 
        value="California",
        className="dbc"
    ),
    dcc.Dropdown(id="city-dropdown", value="Los Angeles", className="dbc"),
    dbc.Card(id="text-output")
])

@app.callback(
    Output("city-dropdown", "options"), 
    Input("state-dropdown", "value"))
def set_metrics_options(selected_state):
    return states_cities[selected_state]

@app.callback(
    Output("text-output", "children"),
    Input("city-dropdown", "value"))
def plot_bar(city):
    if not city:
        raise PreventUpdate
    return f"You have selected {city}!"

app.run_server(debug=True, port=8170)

Dash app running on http://127.0.0.1:8170/


In [3]:
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template

import plotly.express as px
import pandas as pd
import numpy as np

dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SLATE, dbc_css])

load_figure_template("SLATE")

education = (
    pd.read_csv("../Data/states_all.csv")
    .iloc[:, 1:]
    .assign(
        Avg_Math = lambda x: x[["AVG_MATH_4_SCORE", "AVG_MATH_8_SCORE"]].mean(axis=1),
        Avg_Reading = lambda x: x[["AVG_READING_4_SCORE", "AVG_READING_8_SCORE"]].mean(axis=1),
    )
)

all_options = {
    "By Subject": [
        {"label": "Math", "value": "Avg_Math" },
        {"label": "Reading", "value": "Avg_Reading"}
    ],
    "By Grade and Subject": [
        {"label": "4th Grade Math", "value": "AVG_MATH_4_SCORE" },
        {"label": "8th Grade Math", "value": "AVG_MATH_8_SCORE" },
        {"label": "4th Grade Reading", "value": "AVG_READING_4_SCORE" },
        {"label": "8th Grade Reading", "value": "AVG_READING_8_SCORE" },
    ],
}

app.layout= html.Div([
    dcc.RadioItems(
        id="report-type-radio", 
        options=["By Subject", "By Grade and Subject"], 
        value="By Grade and Subject"
    ),
    dcc.RadioItems(id="metric-radio"),
    dcc.Graph(id="metric-bar")
])

@app.callback(
    Output("metric-radio", "options"), 
    Input("report-type-radio", "value"))
def set_metrics_options(selected_report):
    return all_options[selected_report]

@app.callback(
    Output("metric-bar", "figure"),
    Input("metric-radio", "value"))
def plot_bar(metric):
    if not metric:
        raise PreventUpdate
    figure = px.bar((
        education
        .groupby("STATE", as_index=False)
        .agg({metric: "mean"})
        .sort_values(metric, ascending=False)
        ),
        x="STATE",
        y=metric
    )
    return figure

app.run_server(debug=True, port=8171)

Dash app running on http://127.0.0.1:8171/


### Cross-Filtering

In [14]:
from jupyter_dash import JupyterDash
from dash import dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template

import plotly.express as px
import pandas as pd
import numpy as np

dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SLATE, dbc_css])

load_figure_template("SLATE")

education = (pd
             .read_csv("../Data/states_all.csv")
             .iloc[:, 1:]
             .rename({
                 "AVG_MATH_4_SCORE": "4th Gr. Math",
                 "AVG_MATH_8_SCORE": "8th Gr. Math",
                 "AVG_READING_4_SCORE": "4th Gr. Reading",
                 "AVG_READING_8_SCORE": "8th Gr. Reading"
             }, axis=1)
             .assign(expenditure_per_student=lambda x: x["TOTAL_EXPENDITURE"]/x["GRADES_ALL_G"])
)


app.layout = html.Div([
    dbc.Row(html.H1("Education Performance and Expenditure in the US", style={"text-align": "center"})),
    dbc.Row([
       dbc.Col([
           dbc.Card([
           dcc.Markdown("Select an X Column"),
           dcc.RadioItems(
               id="score-radio",
               options=["4th Gr. Math", "8th Gr. Math", "4th Gr. Reading", "8th Gr. Reading"], 
               value="8th Gr. Math"
           ),
           html.Hr(),
           dcc.Markdown("Select a Y Column"),
           dcc.RadioItems(
                id="score-radio2",
                options=["4th Gr. Math", "8th Gr. Math", "4th Gr. Reading", "8th Gr. Reading"],
                value="8th Gr. Reading"
           )])
       ], width=2),
        dbc.Col(
            dcc.Graph(id="cross-filter-scatter", hoverData={'points': [{'customdata': ['CALIFORNIA']}]})
        ),
        dbc.Col(dcc.Graph(id="x-line"))
    ])
])
@app.callback(
    Output("cross-filter-scatter", "figure"),
    Input("score-radio", "value"),
    Input("score-radio2", "value"))
def score_scatter(x, y):
    fig = px.scatter(
        education.query("YEAR == 2013"),
        x=x,
        y=y,
        hover_name="STATE",
        custom_data=["STATE"]
    )
        
    return fig

    
@app.callback(
    Output("x-line", "figure"),
    Input("cross-filter-scatter", "hoverData"))
def update_line(hoverData):
    state_name = hoverData["points"][0]["customdata"][0]
    df=education.query("STATE == @state_name")
    
    fig = px.line(
        df,
        x="YEAR",
        y="expenditure_per_student",
        title=f"Expenditure Per Student in {state_name.title()}"
    ).update_xaxes(showgrid=False)
    
    return fig
    

if __name__ == "__main__":
    app.run_server(debug=True, port=8256)


Dash app running on http://127.0.0.1:8256/


### Manually Fired Callbacks

In [16]:
from jupyter_dash import JupyterDash
from dash import dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template

import plotly.express as px
import pandas as pd
import numpy as np

dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SLATE, dbc_css])

load_figure_template("SLATE")

education = (pd
             .read_csv("../Data/states_all.csv")
             .iloc[:, 1:]
             .rename({
                 "AVG_MATH_4_SCORE": "4th Gr. Math",
                 "AVG_MATH_8_SCORE": "8th Gr. Math",
                 "AVG_READING_4_SCORE": "4th Gr. Reading",
                 "AVG_READING_8_SCORE": "8th Gr. Reading"
             }, axis=1)
             .assign(expenditure_per_student= lambda x: x["TOTAL_EXPENDITURE"]/x["GRADES_ALL_G"])
)


app.layout = html.Div([
    dbc.Row(html.H1("Education Performance and Expenditure in the US", style={"text-align": "center"})),
    dbc.Row([
       dbc.Col([
           dbc.Card([
               dcc.Markdown("Select an X Column"),
               dcc.RadioItems(
                   id="score-radio",
                   options=["4th Gr. Math", "8th Gr. Math", "4th Gr. Reading", "8th Gr. Reading"], 
                   value="8th Gr. Math"
               ),
               html.Hr(),
               dcc.Markdown("Select a Y Column"),
               dcc.RadioItems(
                    id="score-radio2",
                    options=["4th Gr. Math", "8th Gr. Math", "4th Gr. Reading", "8th Gr. Reading"],
                    value="8th Gr. Reading"
               ),
               html.Button("Submit", id="submit-button")])
       ], width=2),
        dbc.Col(dcc.Graph(id="cross-filter-scatter", 
                          hoverData={'points': [{'customdata': ['CALIFORNIA']}]}), width=5),
        dbc.Col(dcc.Graph(id="x-line"), width=5)
    ])
])

@app.callback(
    Output("cross-filter-scatter", "figure"),
    Output("submit-button", "n_clicks"),
    Input("score-radio", "value"),
    Input("score-radio2", "value"),
    Input("submit-button", "n_clicks")
)
def score_scatter(x, y, n_clicks):
    if not n_clicks:
        raise PreventUpdate
    fig = px.scatter(
        education.query("YEAR == 2013"),
        x=x,
        y=y,
        hover_name="STATE",
        custom_data=["STATE"],
        title=f"{n_clicks}"
    )        
    n_clicks = None
    return fig, n_clicks


def create_line_chart(df, state_name):
    
    fig = px.line(
        df, x="YEAR", 
        y="expenditure_per_student",
        title=f"Expenditure Per Student in {state_name.title()}"
    )
        
    fig.update_xaxes(showgrid=False)
    
    return fig
    
@app.callback(
    Output("x-line", "figure"),
    Input("cross-filter-scatter", "hoverData"))
def update_line(hoverData):
    state_name = hoverData["points"][0]["customdata"][0]
    df=education.query("STATE == @state_name")
    return create_line_chart(df, state_name)
    

if __name__ == "__main__":
    app.run_server(debug=True,mode="inline", port=8260)


### Periodic Callbacks

In [17]:
from jupyter_dash import JupyterDash
from dash import dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template

import plotly.express as px
import pandas as pd


dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SLATE, dbc_css])

load_figure_template("SLATE")

from numpy.random import default_rng
rng = default_rng(2023)

app.layout = html.Div([
    dbc.Row(html.H1("Normal Distribution Simulator", style={"text-align": "center"})),
    dbc.Row(dbc.Col(dcc.Graph(id="random-data-scatter"))),
    dcc.Interval(id="refresh-data-interval", interval=1000, n_intervals=0)
])

@app.callback(
    Output("random-data-scatter", "figure"),
    Input("refresh-data-interval", "n_intervals")
)

def rand_hist(n_intervals):
    mean, stddev = 100, 10
    fig = px.histogram(
        x=rng.normal(mean, stddev, size=100),
        title=f"Random Histogram has run {n_intervals} times"
    )
    return fig


if __name__ == "__main__":
    app.run_server(debug=True, mode="inline", port=8260)

# Real time Data Via Finnhub API

*Note this won't work unless you get your own API key. The good news is that it's free!*

In [ ]:
# !pip install finnhub-python

In [ ]:
# my_api_key = "YOUR API KEY HERE" 

In [ ]:
import finnhub
finnhub_client = finnhub.Client(api_key=my_api_key)

print(finnhub_client.quote('AAPL'))

In [ ]:
from jupyter_dash import JupyterDash
from dash import dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template

from datetime import datetime, date
import finnhub

import plotly.express as px
import pandas as pd




dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SLATE, dbc_css])


finnhub_client = finnhub.Client(api_key=my_api_key)

load_figure_template("SLATE")

stock_df = pd.DataFrame(finnhub_client.quote('AAPL'), index=[0])

def append_row(dataframe):
    row = finnhub_client.quote('AAPL')
    temp_df = pd.DataFrame(row, index=[0])
    merged_df = pd.concat([dataframe, temp_df])
    return merged_df


app.layout = html.Div([
    dbc.Row(html.H1("Real-Time Stock Tracker", style={"text-align": "center"})),
    dbc.Row(dbc.Col(dcc.Graph(id="stock-price-line"))),
    dcc.Interval(id="refresh-data-interval", interval=1000)
])

@app.callback(
    Output("stock-price-line", "figure"),
    Input("refresh-data-interval", "n_intervals")
)


def stock_line(n_intervals):
    global stock_df
    stock_df = append_row(stock_df)
    
    fig = px.line(
        (stock_df
         .assign(t = pd.to_datetime(stock_df["t"], unit="s"))
         .rename({"c": "Price"}, axis=1)
        ),
        x="t",
        y="Price",
        title=f"AAPL Stock Price as of {datetime.now()}"
    ).update_layout({"title": {"x": .5}})
    return fig


if __name__ == "__main__":
    app.run_server(debug=True, port=8260)
